## 141: Implementar Autenticación Segura con JWT (JSON Web Token) en FastAPI

Hoy vas a reemplazar el token estático por autenticación con JWT (JSON Web Token), lo que permite:

✅ Autenticación más segura y escalable

🔁 Inicios de sesión con expiración

🧩 Uso futuro con múltiples usuarios y roles

🧩 1. Instalar librerías necesarias
bash
Copiar
Editar
pip install python-jose[cryptography] passlib[bcrypt]
🧩 2. Crear archivo auth.py con funciones de autenticación
python
Copiar
Editar
# auth.py
from datetime import datetime, timedelta
from jose import JWTError, jwt
from fastapi import HTTPException, Depends
from fastapi.security import OAuth2PasswordBearer

# Configuración del token
SECRET_KEY = "clave_secreta_super_segura"
ALGORITHM = "HS256"
EXPIRE_MINUTES = 30

# Ruta de login protegida por este esquema
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

def crear_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.utcnow() + (expires_delta or timedelta(minutes=EXPIRE_MINUTES))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

def verificar_token(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except JWTError:
        raise HTTPException(status_code=403, detail="Token inválido o expirado")
🧩 3. Actualizar main.py para usar JWT
python
Copiar
Editar
from fastapi import FastAPI, Depends, HTTPException, Form
from auth import crear_token, verificar_token
from fastapi.security import OAuth2PasswordRequestForm
from pydantic import BaseModel
from report_utils import crear_pdf_desde_dataframe, enviar_pdf_por_correo
import pandas as pd

app = FastAPI()

# Simulación de usuario único (puedes usar base de datos real)
USUARIO = {"username": "admin", "password": "1234", "email": "admin@correo.com"}

# Login y generación de token
@app.post("/login")
def login(form_data: OAuth2PasswordRequestForm = Depends()):
    if form_data.username != USUARIO["username"] or form_data.password != USUARIO["password"]:
        raise HTTPException(status_code=401, detail="Credenciales incorrectas")
    
    token = crear_token({"sub": form_data.username})
    return {"access_token": token, "token_type": "bearer"}

# Modelo de entrada para el reporte
class ReporteRequest(BaseModel):
    email: str
    data: list

# Ruta protegida por JWT
@app.post("/enviar-reporte/")
def enviar_reporte(data: ReporteRequest, user: dict = Depends(verificar_token)):
    try:
        df = pd.DataFrame(data.data)
        if df.empty:
            raise HTTPException(status_code=400, detail="Datos vacíos.")

        pdf = crear_pdf_desde_dataframe(df)
        enviar_pdf_por_correo(data.email, pdf)
        return {"status": "enviado", "email": data.email}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
🧪 4. Cómo probar con cURL o Postman
Paso 1: Obtener el token
bash
Copiar
Editar
curl -X POST http://localhost:8000/login \
-F 'username=admin' -F 'password=1234'
Obtendrás:

json
Copiar
Editar
{
  "access_token": "eyJhbGciOiJIUzI1NiIsInR5cCI6...",
  "token_type": "bearer"
}
Paso 2: Usar el token para enviar el reporte
bash
Copiar
Editar
curl -X POST http://localhost:8000/enviar-reporte/ \
-H "Authorization: Bearer TU_TOKEN_AQUI" \
-H "Content-Type: application/json" \
-d '{"email":"user@correo.com", "data":[{"Fecha":"2024-01-01","Tipo":"PDF","Categoría":"Diario"}]}'
